In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

---
**Préparation du jeu de données**

In [26]:
df = pd.read_csv("results.csv")

In [27]:
df = df.loc[df["tournament"] == "African Cup of Nations"]

In [28]:
df.drop(columns=["tournament"], inplace=True)

In [29]:
df.head(5)

,date,home_team,away_team,home_score,away_score,city,country,neutral
4099,1957-02-10,Sudan,Egypt,1,2,Khartoum,Sudan,False
4100,1957-02-16,Egypt,Ethiopia,4,0,Khartoum,Sudan,True
4539,1959-05-22,Egypt,Ethiopia,4,0,Cairo,United Arab Republic,True
4542,1959-05-25,Ethiopia,Sudan,0,1,Cairo,United Arab Republic,True
4545,1959-05-29,Egypt,Sudan,2,1,Cairo,United Arab Republic,True


---
**Feature Engineering pour voir le nombre de victoires et de défaites par équipe**
On traitera le cas des matchs nuls plus tard

In [30]:
# Si l'équipe à domicile a gagné, la colonne home_win sera à 1 sinon away_win sera à 0
df['home_win'] = df['home_score'] > df['away_score']
df['away_win'] = df['away_score'] > df['home_score']

In [31]:
# Si l'équipe à domicile a gagné, la colonne home_win sera à 1 sinon away_win sera à 0
df['home_loss'] = df['home_score'] < df['away_score']
df['away_loss'] = df['away_score'] < df['home_score']

In [32]:
total_wins = pd.concat([df.groupby('home_team')['home_win'].sum(),
                        df.groupby('away_team')['away_win'].sum()], axis=1, sort=False).sum(axis=1)
total_wins

Algeria              28
Angola                4
Benin                 0
Botswana              0
Burkina Faso          9
Burundi               0
Cameroon             45
Cape Verde            2
Comoros               1
Congo                 7
DR Congo             20
Egypt                60
Equatorial Guinea     6
Ethiopia              7
Gabon                 7
Gambia                3
Ghana                54
Guinea               13
Guinea-Bissau         0
Ivory Coast          44
Kenya                 2
Liberia               1
Libya                 3
Madagascar            2
Malawi                2
Mali                 19
Mauritania            0
Mauritius             0
Morocco              27
Mozambique            0
Namibia               0
Niger                 0
Nigeria              53
Rwanda                1
Senegal              27
Sierra Leone          1
South Africa         16
Sudan                 7
Tanzania              0
Togo                  3
Tunisia              25
Uganda          

In [34]:
total_loss = pd.concat([df.groupby('home_team')['home_loss'].sum(),
                        df.groupby('away_team')['away_loss'].sum()], axis=1, sort=False).sum(axis=1)
total_loss

Algeria              27
Angola               10
Benin                 9
Botswana              3
Burkina Faso         23
Burundi               3
Cameroon             16
Cape Verde            3
Comoros               3
Congo                12
DR Congo             29
Egypt                27
Equatorial Guinea     5
Ethiopia             19
Gabon                 8
Gambia                1
Ghana                27
Guinea               17
Guinea-Bissau         6
Ivory Coast          28
Kenya                11
Liberia               2
Libya                 3
Madagascar            1
Malawi                6
Mali                 16
Mauritania            4
Mauritius             3
Morocco              19
Mozambique           10
Namibia               7
Niger                 5
Nigeria              22
Rwanda                1
Senegal              23
Sierra Leone          4
South Africa         14
Sudan                13
Tanzania              5
Togo                 14
Tunisia              26
Uganda          

---
**Tracer la courbe pour les 10 équipes avec le plus de victoires**

In [42]:
# Trier les équipes en fonction du nombre total de victoires
total_wins = total_wins.sort_values(ascending=False)

# Sélectionner les 10 premiers résultats
top_10_wins = total_wins.head(10)

# Définir des couleurs personnalisées pour chaque barre
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'lightsalmon',
          'lightsteelblue', 'lightpink', 'lightseagreen', 'lightcyan', 'yellow']

# Tracer le graphique avec Plotly
fig = px.bar(top_10_wins, x=top_10_wins.index, y=top_10_wins.values, color=top_10_wins.index,
             color_discrete_sequence=colors, labels={'index': 'Équipe', 'y': 'Nombre total de victoires'})

# Ajouter le texte au-dessus de chaque barre
for i, value in enumerate(top_10_wins.values):
    fig.add_annotation(x=i, y=value, text=str(value), showarrow=False, font=dict(size=8))

# Personnaliser la disposition
fig.update_layout(
    title='Top 10 des équipes avec le plus grand nombre de victoires',
    xaxis_title='Équipe',
    yaxis_title='Nombre total de victoires',
    xaxis=dict(tickangle=-45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_wins.index)
)

# Afficher le graphique
fig.show()

---
**Tracer la courbe pour les 10 équipes avec le plus de défaites**

In [43]:
# Trier les équipes en fonction du nombre total de victoires
total_loss = total_loss.sort_values(ascending=False)

# Sélectionner les 10 premiers résultats
top_10_loss = total_loss.head(10)

# Définir des couleurs personnalisées pour chaque barre
colors = ['skyblue', 'lightcoral', 'lightgreen', 'gold', 'lightsalmon',
          'lightsteelblue', 'lightpink', 'lightseagreen', 'lightcyan', 'yellow']

# Tracer le graphique avec Plotly
fig = px.bar(top_10_loss, x=top_10_loss.index, y=top_10_loss.values, color=top_10_loss.index,
             color_discrete_sequence=colors, labels={'index': 'Équipe', 'y': 'Nombre total de défaites'})

# Ajouter le texte au-dessus de chaque barre
for i, value in enumerate(top_10_loss.values):
    fig.add_annotation(x=i, y=value, text=str(value), showarrow=False, font=dict(size=8))

# Personnaliser la disposition
fig.update_layout(
    title='Top 10 des équipes avec le plus grand nombre de défaites',
    xaxis_title='Équipe',
    yaxis_title='Nombre total de défaites',
    xaxis=dict(tickangle=-45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_loss.index)
)

# Afficher le graphique
fig.show()

**Nous allons refaire le même travail en se basant sur les pourcentages**

In [44]:
# Nombre total de matches par équipe
total_games = pd.concat([df['home_team'].value_counts(),
                         df['away_team'].value_counts()], axis=1, sort=False).sum(axis=1)

In [47]:
# Pourcentage de victoires
win_percentage = (total_wins / total_games) * 100

# Les 10 équipes avec les plus grands pourcentages de victoires
top_10_win_percentage = win_percentage.nlargest(10).reset_index()
top_10_win_percentage.columns = ['Équipe', 'Pourcentage de victoires']

top_10_win_percentage['Pourcentage de victoires'] = top_10_win_percentage['Pourcentage de victoires'].round(2)

# Tracer le graphique avec Plotly
fig = px.bar(top_10_win_percentage, x='Équipe', y='Pourcentage de victoires',
             title='Top 10 des équipes avec le plus haut pourcentage de victoires',
             labels={'Pourcentage de victoires': 'Pourcentage de victoires'},
             text='Pourcentage de victoires',
             height=500, color='Équipe', color_discrete_sequence=colors)

# Personnaliser l'apparence
fig.update_traces(marker_line_color='black', marker_line_width=1.5, textposition='outside')
fig.update_layout(xaxis=dict(tickangle=45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_win_percentage['Équipe']))

# Afficher le graphique
fig.show()

In [50]:
# Pourcentage de défaites
loss_percentage = (total_loss / total_games) * 100

# Les 10 équipes avec les plus grands pourcentages de défaites
top_10_loss_percentage = loss_percentage.nlargest(10).reset_index()
top_10_loss_percentage.columns = ['Équipe', 'Pourcentage de défaites']

top_10_loss_percentage['Pourcentage de défaites'] = top_10_loss_percentage['Pourcentage de défaites'].round(2)

# Tracer le graphique avec Plotly
fig = px.bar(top_10_loss_percentage, x='Équipe', y='Pourcentage de défaites',
             title='Top 10 des équipes avec le plus haut pourcentage de défaites',
             labels={'Pourcentage de défaites': 'Pourcentage de défaites'},
             text='Pourcentage de défaites',
             height=500, color='Équipe', color_discrete_sequence=colors)

# Personnaliser l'apparence
fig.update_traces(marker_line_color='black', marker_line_width=1.5, textposition='outside')
fig.update_layout(xaxis=dict(tickangle=45, tickmode='array', tickvals=list(range(10)), ticktext=top_10_loss_percentage['Équipe']))

# Afficher le graphique
fig.show()